In [96]:
import pandas as pd

# Data Exploration



### Assumptions
- Train is delayed if delay is more than 5 Minutes
- Train data consists only of train departures where planned and changeddeparture time is available
- Map of weather data to train station with name because there is (currently) no geo data available from train staions
( Idea: Geographical data analysis https://techlabsdus.medium.com/analysis-of-deutsche-bahn-ag-long-distance-train-traffic-delays-b183aacad9a2
To avoid further work with different API, all longitudes and latitudes for the main train stations were taken from the txt.files from GTFS.de (4.c.) by converting to csv-files and data frames. For a bigger project API’s should be taken into consideration.
)
- Restriction to main stations in germany 


## Load weather data

In [97]:
weather_stations= pd.read_sql_table('weather_stations','sqlite:///data/amse.sqlite')
weather_stations

,Unnamed: 0,id,name,country,region,identifiers,location,timezone,inventory
0,1227,10004,{'en': 'UFS TW Ems'},DE,HH,"{'national': '01228', 'wmo': '10004', 'icao': ...","{'latitude': 54.1667, 'longitude': 6.35, 'elev...",Europe/Berlin,"{'model': {'start': '2018-01-28', 'end': '2023..."
1,1228,10007,{'en': 'UFS Deutsche Bucht.'},DE,HH,"{'national': '00954', 'wmo': '10007', 'icao': ...","{'latitude': 54.1833, 'longitude': 7.4333, 'el...",Europe/Berlin,"{'model': {'start': '2018-01-28', 'end': '2023..."
2,1229,10008,{'en': 'Nordseeboje 2'},DE,HH,"{'national': '13904', 'wmo': '10008', 'icao': ...","{'latitude': 55.0, 'longitude': 6.3333, 'eleva...",Europe/Berlin,"{'model': {'start': None, 'end': None}, 'hourl..."
3,1230,10015,{'en': 'Helgoland'},DE,SH,"{'national': '02115', 'wmo': '10015', 'icao': ...","{'latitude': 54.1833, 'longitude': 7.9, 'eleva...",Europe/Berlin,"{'model': {'start': '2018-01-28', 'end': '2023..."
4,1231,10018,{'en': 'Westerland / Sylt'},DE,SH,"{'national': None, 'wmo': '10018', 'icao': 'ED...","{'latitude': 54.9167, 'longitude': 8.35, 'elev...",Europe/Berlin,"{'model': {'start': '2018-01-28', 'end': '2023..."
...,...,...,...,...,...,...,...,...,...
1179,15583,XXIO0,"{'en': 'Bretten, Kreis Karlsruhe'}",DE,BW,"{'national': '07490', 'wmo': None, 'icao': None}","{'latitude': 49.0334, 'longitude': 8.7121, 'el...",Europe/Berlin,"{'model': {'start': '2023-05-13', 'end': '2023..."
1180,15589,Y4O2A,{'en': 'Backnang/murr'},DE,BW,"{'national': '07380', 'wmo': None, 'icao': None}","{'latitude': 48.9353, 'longitude': 9.4009, 'el...",Europe/Berlin,"{'model': {'start': '2023-05-13', 'end': '2023..."
1181,15594,YA9TW,"{'en': 'Boll, Bad (kläranl.)'}",DE,BW,"{'national': '05985', 'wmo': None, 'icao': None}","{'latitude': 48.652, 'longitude': 9.6232, 'ele...",Europe/Berlin,"{'model': {'start': '2023-05-14', 'end': '2023..."
1182,15618,YJQ5W,{'en': 'Enzklösterle'},DE,BW,"{'national': '05840', 'wmo': None, 'icao': None}","{'latitude': 48.6671, 'longitude': 8.47, 'elev...",Europe/Berlin,"{'model': {'start': '2023-05-14', 'end': '2023..."


In [98]:
weather= pd.read_sql_table('weather','sqlite:///data/amse.sqlite')
weather

,date,hour,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,station
0,2023-01-02,0,13.3,8.4,72.0,NaN,NaN,196.0,27.8,42.6,1012.4,NaN,4.0,10501
1,2023-01-02,1,13.3,8.6,73.0,NaN,NaN,200.0,27.8,44.5,1012.5,NaN,4.0,10501
2,2023-01-02,2,13.2,8.7,74.0,NaN,NaN,200.0,29.6,44.5,1012.5,NaN,4.0,10501
3,2023-01-02,3,13.0,8.9,76.0,NaN,NaN,202.0,29.6,46.3,1012.5,NaN,7.0,10501
4,2023-01-02,4,12.6,8.9,78.0,NaN,NaN,205.0,25.9,44.5,1012.4,NaN,7.0,10501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768305,2023-06-06,14,21.1,8.7,45.0,NaN,NaN,19.0,13.0,27.8,1016.1,NaN,2.0,10714
768306,2023-06-06,15,21.1,8.7,45.0,NaN,NaN,22.0,13.0,27.8,1015.9,NaN,2.0,10714
768307,2023-06-06,16,20.8,8.8,46.0,NaN,NaN,17.0,13.0,27.8,1015.8,NaN,2.0,10714
768308,2023-06-06,17,20.5,11.4,56.0,NaN,NaN,291.0,13.0,25.9,1015.5,NaN,3.0,10714


## Load train data

In [99]:
train_stations= pd.read_sql_table('train_stations','sqlite:///data/amse.sqlite')
train_stations

,name,eva,ds100,db,creationts,meta,updatets
0,Aachen Hbf,8000001,KA,True,23-05-25 11:02:42.341,None,None
1,Aalen Hbf,8000002,TA,True,23-05-25 11:02:42.341,None,None
2,Arnstadt Hbf,8010007,UA,True,23-05-25 11:02:42.258,None,None
3,Aschaffenburg Hbf,8000010,NAH,True,23-05-25 11:02:42.341,None,None
4,Augsburg Hbf,8000013,MA,True,23-05-25 11:02:42.341,None,None
...,...,...,...,...,...,...,...
192,Würzburg Hbf,8000260,NWH,True,23-05-25 11:02:42.345,8089299,None
193,Würzburg Hbf (Busbahnhof),8089299,NWH H,True,23-05-25 11:02:42.405,8000260,None
194,Zweibrücken Hbf,8006680,SZW,True,23-05-25 11:02:42.269,None,None
195,Zwickau(Sachs)Hbf,8010397,DZW,True,23-05-25 11:02:42.163,None,None


In [100]:
timetables= pd.read_sql_table('timetables3005','sqlite:///data/amse.sqlite')
timetables

,eva,ct,pt
0,8000001,"29.05.2023, 22:09","29.05.2023, 21:39"
1,8000001,"29.05.2023, 22:09","29.05.2023, 21:39"
2,8000001,"29.05.2023, 22:09","29.05.2023, 21:39"


## Load match data

In [101]:
match_table= pd.read_sql_table('match_table','sqlite:///data/amse.sqlite')
match_table

,eva,name_x,common,id,name_y,match
0,8000001,Aachen Hbf,Aachen,10501,{'en': 'Aachen'},True
1,8000001,Aachen Hbf,Aachen,10503,{'en': 'Aachen / Merzbruck'},True
2,8000013,Augsburg Hbf,Augsburg,10852,{'en': 'Augsburg'},True
3,8000028,Bayreuth Hbf,Bayreuth,10677,{'en': 'Bayreuth'},True
4,8000885,Berchtesgaden Hbf,Berchtesgaden,D6190,{'en': 'Berchtesgaden'},True
...,...,...,...,...,...,...
227,8000379,Wittlich Hbf,Wittlich,D5635,{'en': 'Wittlich'},True
228,8006552,Wolfsburg Hbf,Wolfsburg,D5676,{'en': 'Wolfsburg (Südwest)'},True
229,8000257,Worms Hbf,Worms,EDFV0,{'en': 'Worms / Nonnenhof'},True
230,8000266,Wuppertal Hbf,Wuppertal,D5717,{'en': 'Wuppertal-Buchenhofen'},True


### Plot count of delayed trains at station

### Plot avergare temp per day at station

### Plot correlation between delayed trains and temp

### Plot map of train delays frequencies in germany